In [2]:
import pandas as pd
from pandasql import sqldf
from pandasql import *
pysqldf = lambda q: sqldf(q, globals())

In [3]:
train_full = pd.read_csv('train.csv',delimiter='|')

In [4]:
train_full.head()

,date,pid,size,units
0,2017-10-01,14393,2 ( 37-39 ),1
1,2017-10-01,10069,36,2
2,2017-10-01,10069,35,1
3,2017-10-01,16221,L,1
4,2017-10-01,11317,L,1


In [5]:
item=pd.read_csv('items.csv',delimiter='|')

In [6]:
train_full['key']=train_full['pid'].astype(str) +" "+train_full['size'].astype(str)
item['key']=item['pid'].astype(str) +" "+item['size'].astype(str)

In [7]:
item.count()

pid             12824
size            12815
color           12824
brand           12824
rrp             12824
mainCategory    12824
category        12824
subCategory     11690
stock           12824
releaseDate     12824
key             12824
dtype: int64

In [8]:
train_full['date'] = pd.to_datetime(train_full['date'], infer_datetime_format=True)

In [9]:
train_full.head()

,date,pid,size,units,key
0,2017-10-01,14393,2 ( 37-39 ),1,14393 2 ( 37-39 )
1,2017-10-01,10069,36,2,10069 36
2,2017-10-01,10069,35,1,10069 35
3,2017-10-01,16221,L,1,16221 L
4,2017-10-01,11317,L,1,11317 L


In [10]:
train_full.count()

date     135117
pid      135117
size     134941
units    135117
key      135117
dtype: int64

### low sales <=2

In [11]:
q  = "select key,sum(units) as total_sales from train_full where date<='2017-12-31' group by key having sum(units)<=2;"
train_low_sales = pysqldf(q)

In [12]:
train_low_sales.count()

key            3446
total_sales    3446
dtype: int64

In [13]:
train_low_sales['category']=0
train_low_sales.head()

,key,total_sales,category
0,10000 XL ( 158-170 ),1,0
1,10001 L,2,0
2,10006 XL,1,0
3,10008 XL,2,0
4,10013 M,1,0


### avg sales >2 & <=10

In [14]:
q  = "SELECT key,sum(units) as total_sales from train_full where date<='2017-12-31' group by key having sum(units)>2 and sum(units)<=10;"
train_avg_sales = pysqldf(q)

In [15]:
train_avg_sales.count()

key            4228
total_sales    4228
dtype: int64

In [16]:
train_avg_sales['category']=1
train_avg_sales.head()

,key,total_sales,category
0,10003 4 ( 39-42 ),3,1
1,10003 5 ( 43-46 ),10,1
2,10017 L,4,1
3,10020 XL,3,1
4,10035 L ( 152-158 ),8,1


### good sales >10 & <=25

In [17]:
q  = "SELECT key,sum(units) as total_sales from train_full where date<='2017-12-31' group by key having sum(units)>10 and sum(units)<=25;"
train_good_sales = pysqldf(q)

In [18]:
train_good_sales.count()

key            1701
total_sales    1701
dtype: int64

In [19]:
train_good_sales['category']=2
train_good_sales.head()

,key,total_sales,category
0,10003 3 (35-38 ),15,2
1,10062 L,12,2
2,10063 M,25,2
3,10063 XL,21,2
4,10073 XL ( 44/46 ),16,2


### top sales  >25

In [20]:
q  = "SELECT key,sum(units) as total_sales from train_full where date<='2017-12-31' group by key having sum(units)>25;"
train_top_sales = pysqldf(q)

In [21]:
train_top_sales.count()

key            1610
total_sales    1610
dtype: int64

In [22]:
train_top_sales['category']=3
train_top_sales.head()

,key,total_sales,category
0,10063 L,28,3
1,10065 2 ( 37-39 ),122,3
2,10065 4 ( 43-45 ),158,3
3,10073 L ( 40/42 ),32,3
4,10125 L,79,3


### total items count upto dec

In [23]:
q  = "SELECT key,count(*) as total_rows from train_full where date<='2017-12-31' group by key;"
train_rows = pysqldf(q)

In [24]:
train_rows.count()

key           10985
total_rows    10985
dtype: int64

### new items in jan

In [25]:
q  = "SELECT key from item where key not in (select key from train_full where date<='2017-12-31');"
new_in_january = pysqldf(q)

In [26]:
new_in_january.count()

key    1839
dtype: int64

In [27]:
new_in_january['total_sales']=0
new_in_january['category']=4
new_in_january.head()

,key,total_sales,category
0,10013 L,0,4
1,10031 43,0,4
2,10039 46,0,4
3,"10042 37,5",0,4
4,10043 XL,0,4


### append all data frames to final and export to excel

In [28]:
col_names =  ['key', 'total_sales', 'category']
train_discretized_item  = pd.DataFrame(columns = col_names)
train_discretized_item

,key,total_sales,category


In [29]:
train_discretized_item=train_discretized_item.append(train_low_sales,ignore_index=True)
train_discretized_item=train_discretized_item.append(train_avg_sales,ignore_index=True)
train_discretized_item=train_discretized_item.append(train_good_sales,ignore_index=True)
train_discretized_item=train_discretized_item.append(train_top_sales,ignore_index=True)
train_discretized_item=train_discretized_item.append(new_in_january,ignore_index=True)

In [30]:
train_discretized_item.head()

,key,total_sales,category
0,10000 XL ( 158-170 ),1,0
1,10001 L,2,0
2,10006 XL,1,0
3,10008 XL,2,0
4,10013 M,1,0


In [32]:
train_discretized_item.to_csv('discretized_item_by_sales.csv')

In [31]:
train_discretized_item.count()

key            12824
total_sales    12824
category       12824
dtype: int64